Nadia AHMED

# TP Introduction : DynamoDB

In [1]:
import boto3
from boto3.dynamodb.conditions import Key
import requests

dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url='http://localhost:8000')

### Partie 1 : Récupérer les données de l'API iTunes ###

In [2]:
import requests

# Liste des artistes pour lesquels vous souhaitez obtenir des données
artists = ["depeche mode", "queen", "the beatles", "coldplay", "dido"]

# Dictionnaire pour stocker les données des artistes
all_artist_data = {}

# Boucle à travers la liste des artistes
for artist in artists:
    # Construire l'URL de recherche pour l'artiste actuel
    search_url = f"https://itunes.apple.com/search?term={artist}&limit=25"
    
    # Faire la requête HTTP
    response = requests.get(search_url)
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Récupérer les données et les ajouter au dictionnaire
        artist_data = response.json()["results"]
        all_artist_data[artist] = artist_data
        print(f"Données pour l'artiste {artist} récupérées avec succès.")
    else:
        print(f"Échec de la requête pour l'artiste {artist}.")

# Afficher les données pour tous les artistes
for artist, data in all_artist_data.items():
    print(f"\nDonnées pour l'artiste {artist}:")
    print(data)

# Récupérer une variable itunes_data avec les données de tous les artistes
itunes_data = all_artist_data

Données pour l'artiste depeche mode récupérées avec succès.
Données pour l'artiste queen récupérées avec succès.
Données pour l'artiste the beatles récupérées avec succès.
Données pour l'artiste coldplay récupérées avec succès.
Données pour l'artiste dido récupérées avec succès.

Données pour l'artiste depeche mode:
[{'wrapperType': 'audiobook', 'artistId': 284145822, 'collectionId': 329950409, 'artistName': 'Alan Cross', 'collectionName': 'Depeche Mode: The Alan Cross Guide (Unabridged)', 'collectionCensoredName': 'Depeche Mode: The Alan Cross Guide (Unabridged)', 'artistViewUrl': 'https://books.apple.com/us/author/alan-cross/id284145822?uo=4', 'collectionViewUrl': 'https://books.apple.com/us/audiobook/depeche-mode-the-alan-cross-guide-unabridged/id329950409?uo=4', 'artworkUrl60': 'https://is1-ssl.mzstatic.com/image/thumb/Features/v4/84/dc/6a/84dc6a3f-9e22-b417-c261-d5dbc0cec329/dj.yfcrpkwn.jpg/60x60bb.jpg', 'artworkUrl100': 'https://is1-ssl.mzstatic.com/image/thumb/Features/v4/84/dc/

In [3]:
table = dynamodb.create_table(
    TableName='iTunes',
    KeySchema=[
        {
            'AttributeName': 'Artist',
            'KeyType': 'HASH'  
        },
        {
            'AttributeName': 'SongTitle',
            'KeyType': 'RANGE' 
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'Artist',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'SongTitle',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'Album',
            'AttributeType': 'S'
        }
    ],
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'Album-index',
            'KeySchema': [
                {
                    'AttributeName': 'Album',
                    'KeyType': 'HASH'
                }
            ],
            'Projection': {
                'ProjectionType': 'ALL'
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: Cannot create preexisting table

### Partie 2 : Préparer les données ###

#### Transformez les données récupérées pour les insérer dans la table DynamoDB ####

In [ ]:
def prepare_data(item):
    if 'trackName' in item and 'artistName' in item:
        artist_name = item["artistName"]
        song_title = item["trackName"]
        album = item.get("collectionName", "Unknown Album")
        year = item.get("releaseDate", "Unknown Year")[:4]

        return {
            "Artist": artist_name,
            "SongTitle": song_title,
            "Album": album,
            "Year": year
        }
    else:
        return None


all_artist_data = [data for data in all_artist_data if data is not None]
prepared_data = [prepare_data(item) for artist_data in all_artist_data for item in artist_data]
prepared_data = [data for data in prepared_data if data is not None]

for item in prepared_data:
    table.put_item(Item=item)

#### Insérer les éléments dans la table ####

In [ ]:
for item in prepared_data:
    table.put_item(Item=item)

In [ ]:
try:
    # Méthode scan pour obtenir tous les éléments de la table
    response = table.scan()
    items = response['Items']

    # Afficher les éléments de la table
    for item in items:
        print(item)

except Exception as e:
    print(f"Une erreur s'est produite : {e}")

### Partie 3 : Exécuter des requêtes sur la table ###

#### Obtenir toutes les chansons d'un artiste spécifique : 

In [ ]:
response = table.query(
    KeyConditionExpression=Key('Artist').eq('Coldplay')
)
items = response['Items']
print(items)

#### Obtenir toutes les chansons d'un album spécifique :

In [ ]:
response = table.query(
    IndexName='Album-index',
    KeyConditionExpression=Key('Album').eq('The Best of Depeche Mode, Vol. 1 (Deluxe Version)')
)
items = response['Items']
print(items)

#### Obtenir toutes les chansons d'un artiste spécifique sorties après une certaine date :

In [ ]:
from boto3.dynamodb.conditions import Key

response = table.query(
    KeyConditionExpression=Key('Artist').eq('Dido') 
)
filtered_items = [item for item in response['Items'] if 'Year' in item and item['Year'] > '1987']
print(filtered_items)

#### Supprimer une chanson spécifique d'un artiste :

In [ ]:
response = table.delete_item(
    Key={
        'Artist': 'Depeche Mode',
        'SongTitle': 'Enjoy the Silence'
    }
)

In [ ]:
response = table.get_item(
    Key={
        'Artist': 'Depeche Mode',
        'SongTitle': 'Enjoy the Silence'
    }
)

if 'Item' in response:
    print("L'élément existe toujours :", response['Item'])
else:
    print("L'élément a été supprimé avec succès.")

#### Mettre à jour le prix d'une chanson spécifique :

In [ ]:
from decimal import Decimal

response = table.update_item(
    Key={
        'Artist': 'Dido',
        'SongTitle': 'Thank You'
    },
    UpdateExpression="set Price = :p",
    ExpressionAttributeValues={
        ':p': Decimal('0.99'),  # Convertir le float en Decimal
    },
    ReturnValues="UPDATED_NEW"
)

In [ ]:
response = table.get_item(
    Key={
        'Artist': 'Dido',
        'SongTitle': 'Thank You'
    }
)

if 'Item' in response:
    updated_item = response['Item']
    print("L'élément mis à jour :", updated_item)
else:
    print("L'élément n'a pas été trouvé.")


In [ ]:
table = dynamodb.Table('iTunes')
response = table.delete()

# Vérifier si la suppression de la table a réussi
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("La table 'iTunes' a été supprimée avec succès.")
else:
    print("Échec de la suppression de la table 'iTunes'. Veuillez vérifier les erreurs dans la réponse.")